# Weather features :
For each row, the script called the Open-Meteo API (or whichever weather source you configured) using those coordinates.

It fetched weather conditions (precipitation, visibility, temperature, weather code, etc.) at that specific location and time.

Then it stored them in columns: precipitation_mm, visibility_m, weathercode, etc.

In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from timezonefinder import TimezoneFinder
import pytz
from tqdm import tqdm

df = pd.read_csv("mbta_delays_dataset.csv")

df["temperature_C"] = None
df["humidity_%"] = None
df["precipitation_mm"] = None
df["visibility_m"] = None
df["windspeed_kmh"] = None
df["weathercode"] = None
df["weather_time"] = None

tf = TimezoneFinder()

for idx, row in tqdm(df.iterrows(), total=len(df)):
    lat = row.get("latitude")
    lon = row.get("longitude")

    if pd.isna(lat) or pd.isna(lon):
        continue

    try:
        tz_name = tf.timezone_at(lat=lat, lng=lon)
        if tz_name is None:
            tz_name = "UTC"
    except:
        tz_name = "UTC"
    local_tz = pytz.timezone(tz_name)

    now_local = datetime.now(local_tz).replace(tzinfo=None)

    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "hourly": "temperature_2m,relative_humidity_2m,precipitation,visibility,wind_speed_10m,weathercode",
        "timezone": tz_name
    }

    try:
        response = requests.get(url, params=params)
        weather = response.json()

        hourly = pd.DataFrame(weather["hourly"])
        hourly["time"] = pd.to_datetime(hourly["time"])

        target_time = now_local + timedelta(hours=1)
        future_window = hourly[hourly["time"] == target_time]

        if future_window.empty:
            future_window = hourly[hourly["time"] > now_local].head(1)

        if not future_window.empty:
            forecast = future_window.iloc[0]
            df.at[idx, "temperature_C"] = forecast["temperature_2m"]
            df.at[idx, "humidity_%"] = forecast["relative_humidity_2m"]
            df.at[idx, "precipitation_mm"] = forecast["precipitation"]
            df.at[idx, "visibility_m"] = forecast["visibility"]
            df.at[idx, "windspeed_kmh"] = forecast["wind_speed_10m"]
            df.at[idx, "weathercode"] = forecast["weathercode"]
            df.at[idx, "weather_time"] = forecast["time"]

    except Exception as e:
        print(f"Weather fetch error at row {idx}: {e}")

df = df[df["actual_arrival"].notna() & (df["actual_arrival"].astype(str).str.strip() != "")]

df.to_csv("mbta_delays_with_weather.csv", index=False)
print("Weather forecast (1 hr ahead) added and cleaned dataset saved!")
print(df.head())



  0%|                                                | 0/146 [00:00<?, ?it/s]


  1%|▎                                       | 1/146 [00:01<04:24,  1.83s/it]


  1%|▌                                       | 2/146 [00:03<03:55,  1.64s/it]


  2%|▊                                       | 3/146 [00:04<03:55,  1.65s/it]


  3%|█                                       | 4/146 [00:06<03:47,  1.60s/it]

...


 99%|█████████████████████████████████████▍| 144/146 [02:25<00:03,  1.65s/it]


 99%|█████████████████████████████████████▋| 145/146 [02:27<00:01,  1.66s/it]


100%|██████████████████████████████████████| 146/146 [02:29<00:00,  1.70s/it]


100%|██████████████████████████████████████| 146/146 [02:29<00:00,  1.02s/it]

Weather forecast (1 hr ahead) added and cleaned dataset saved!
            trip_id     stop_id  vehicle_id    route_id_x arrival_time  \
0  Base-772227-5242    BNT-0000      1718.0  CR-Haverhill     14:00:00   
1  Base-772228-5243   WR-0120-S      1702.0  CR-Haverhill     13:48:00   
2  Base-772228-5243   WR-0163-S      1702.0  CR-Haverhill     13:54:00   
3  Base-772228-5243  WR-0205-02      1702.0  CR-Haverhill     14:01:00   
4  Base-772228-5243  WR-0228-02      1702.0  CR-Haverhill     14:06:00   

              actual_arrival  delay_seconds  scheduled_minutes  \
0  2025-11-08 13:56:57-05:00            0.0              840.0   
1  2025-11-08 13:54:45-05:00            0.0              828.0   
2  2025-11-08 13:59:54-05:00            0.0              834.0   
3  2025-11-08 14:05:30-05:00            0.0              841.0   
4  2025-11-08 14:08:29-05:00            0.0              846.0   

   actual_minutes  delay_minutes  ...  hour_of_day  day_of_week is_peak  \
0      836.950000   

In [4]:
pip install flask

Note: you may need to restart the kernel to use updated packages.


# Engineering Features — One by One


-> Time Features

1. hour_of_day
	Already present in your dataset.
	Extracted from actual_arrival timestamp (Boston local time).
	Example: bus arriving at 14:35 → hour_of_day = 14.
	Why useful: Delays behave differently at rush hours vs midnight.

2. day_of_week
	Already present in your dataset.
	Extracted from actual_arrival (0=Monday, 6=Sunday).
	Example: arrival on Friday → day_of_week = 4.
	Why useful: Weekdays vs weekends have very different congestion.

3. is_peak
	Already present in your dataset.
	Derived rule-based (you coded earlier): e.g., 7–10 AM & 4–7 PM → peak hours = 1, else 0.
	Why useful: Rush hour is strongly correlated with higher delays.

4. is_weekend
	New. Computed from day_of_week.
	If day_of_week >= 5 (Saturday or Sunday) → 1, else 0.
	Why useful: Weekend traffic patterns differ drastically.

5. is_holiday
	New. Computed from actual_arrival.date.
	Cross-checked with a list of US federal holidays (Massachusetts).
	Example: If 2025-07-04 (Independence Day) → 1.
	Why useful: On holidays, traffic is lighter or bus schedules are modified.


-> Weather Features

1. precipitation_mm
	Already present.
	Pulled from the weather API (e.g., Open-Meteo) for the bus’s latitude/longitude at arrival 	time.
	Value in millimeters.

2. visibility_m
	Already present.
	Also from weather API at the bus’s location.
	Value in meters (e.g., heavy fog → 500 m).

3. weathercode
	Already present.
	Encoded weather condition (0 = clear, 3 = partly cloudy, 61 = rain, 71 = snow, etc.).
	API returns it for each lat/lon and time.


-> Schedule/GTFS Features

1. route_id_x
	Already present.
	Comes from the static GTFS mapping (trip → route).
	Identifies which bus route (e.g., 1, 32, 77, CR-Haverhill).

2. stop_id
	Already present.
	Identifies the stop where the arrival happened.

3. trip_length
	New. Computed from GTFS static stop_times.txt.
	For each trip_id, count the number of stops.
	Example: Trip with 45 stops → trip_length = 45.
	Why useful: More stops = more opportunities for delay accumulation.

4. scheduled_runtime
	New. Computed from GTFS static stop_times.txt.
	For each trip_id: difference between last stop’s scheduled arrival and first stop’s scheduled 	arrival (minutes).
	Example: Trip scheduled 08:00 → 09:05 → scheduled_runtime = 65.
	Why useful: Longer trips tend to accumulate more delay.


-> Optional Feature

1. previous_delay
	New. Computed from within your dataset.
	For each trip_id, we sorted stops in actual order, then shifted delay_minutes.
	Example: If Stop A was 2.5 min late → at Stop B, previous_delay = 2.5.
	Why useful: Delay at one stop tends to carry over to the next.

In [5]:
import pandas as pd
from datetime import datetime

file_path = "mbta_delays_with_weather.csv"
df = pd.read_csv(file_path, parse_dates=["actual_arrival", "weather_time"])

df["is_weekend"] = df["day_of_week"].apply(lambda x: 1 if x >= 5 else 0)

us_federal_holidays_2025 = {
    "2025-01-01", "2025-01-20", "2025-02-17", "2025-05-26",
    "2025-07-04", "2025-09-01", "2025-10-13", "2025-11-11",
    "2025-11-27", "2025-12-25"
}
df["is_holiday"] = df["actual_arrival"].dt.strftime("%Y-%m-%d").isin(us_federal_holidays_2025).astype(int)

trips = pd.read_csv("mbta_gtfs_static/trips.txt", dtype=str)
stop_times = pd.read_csv("mbta_gtfs_static/stop_times.txt", dtype=str)

def hms_to_minutes(hms):
    try:
        h, m, s = map(int, hms.split(":"))
        return h * 60 + m + s / 60.0
    except:
        return None

stop_times["arrival_minutes"] = stop_times["arrival_time"].apply(hms_to_minutes)

trip_stats = (
    stop_times.groupby("trip_id")
    .agg(
        trip_length=("stop_id", "count"),
        scheduled_runtime=("arrival_minutes", lambda x: max(x) - min(x) if len(x.dropna()) > 1 else None),
    )
    .reset_index()
)

df = df.merge(trip_stats, on="trip_id", how="left")

df = df.sort_values(by=["trip_id", "actual_arrival"])
df["previous_delay"] = df.groupby("trip_id")["delay_minutes"].shift(1)

engineering_cols = [
    "hour_of_day", "day_of_week", "is_peak", "is_weekend", "is_holiday",
    "precipitation_mm", "visibility_m", "weathercode",
    "route_id_x", "stop_id", "trip_length", "scheduled_runtime",
    "previous_delay"
]

remaining_cols = [c for c in df.columns if c not in engineering_cols]
df = df[remaining_cols + engineering_cols]

output_file = "mbta_delays_with_all_engineering_features.csv"
df.to_csv(output_file, index=False)

print(df.head(10))


            trip_id  vehicle_id arrival_time            actual_arrival  \
0  Base-772227-5242      1718.0     14:00:00 2025-11-08 13:56:57-05:00   
1  Base-772228-5243      1702.0     13:48:00 2025-11-08 13:54:45-05:00   
2  Base-772228-5243      1702.0     13:54:00 2025-11-08 13:59:54-05:00   
3  Base-772228-5243      1702.0     14:01:00 2025-11-08 14:05:30-05:00   
4  Base-772228-5243      1702.0     14:06:00 2025-11-08 14:08:29-05:00   
5  Base-772228-5243      1702.0     14:13:00 2025-11-08 14:13:49-05:00   
6  Base-772228-5243      1702.0     14:21:00 2025-11-08 14:20:48-05:00   
7  Base-772228-5243      1702.0     14:28:00 2025-11-08 14:23:00-05:00   
8  Base-772245-5344      1703.0     13:49:00 2025-11-08 13:53:33-05:00   
9  Base-772245-5344      1703.0     13:53:00 2025-11-08 13:56:15-05:00   

   delay_seconds  scheduled_minutes  actual_minutes  delay_minutes   latitude  \
0            0.0              840.0      836.950000      -3.050000  42.375034   
1            0.0       

# Final Modeling Plan (Updated)


1. Dataset
	Input file: mbta_delays_with_all_engineering_features.csv
	Features used for ML correction:
	- Time: hour_of_day, day_of_week, is_peak, is_weekend, is_holiday
	- Weather: precipitation_mm, visibility_m, weathercode
	- Schedule/GTFS: route_id_x, stop_id, trip_length, scheduled_runtime
	- Propagation: previous_delay
	Target: delay_minutes (MBTA’s realtime baseline delay).
	Extra feature (external): traffic_delay_minutes from TomTom API (computed via latitude, 	longitude).

	Alerts archive (optional) -
	- Archive_2020-2023.zip (one or more CSVs inside).
	- Contains alert records: alert_id, route_id, stop_id, active_period_start_dt, active_period_end_dt, severity.
	- Used to create alert features per row: alert_any, alert_count, alert_severity_max

2. Feature construction & Preprocessing - 
	Numeric features: impute missing with median.
	Categorical features (route_id_x, stop_id, weathercode): impute missing as "MISSING" and 	One-Hot Encode.
	Ensure consistent treatment of time-series (all times in Boston TZ).

	Alert features (vectorized)
	- Efficient merge between each data row and alerts filtered by route or stop.
	For each row, compute:
	- alert_any (0/1 if any active alert at arrival time),
	- alert_count (unique matching alerts),
	- alert_severity_max (maximum severity).

	-> Numeric: hour_of_day, day_of_week, is_peak, is_weekend, is_holiday, precipitation_mm, visibility_m, trip_length, scheduled_runtime, previous_delay, plus computed alert features.
	-> Categorical: weathercode, route_id_x, stop_id (one-hot encoded).

	Rule adjustments - 
	Compute simple average residual deltas (safe_delta) for:
	- peak vs non-peak,
	- precipitation > threshold,
	- low visibility,
	- alert presence

	Saved to preprocessor.pkl

3. Train/Test Split
	Time-aware split: sort by actual_arrival date-time.
	80% training, 20% testing.
	Why: simulates predicting future from past, avoids leakage.

4. Baseline Model: Linear Regression
	Simple, interpretable model.
	Trains on engineered features to predict residuals.
	Provides a baseline MAE/R² for comparison.

5. Main Model: LightGBM Regressor
	Gradient Boosted Trees (preferred for tabular data).
	Strengths:
	- Handles non-linear relationships (e.g., rain + peak hour).
	- Can handle categorical + numerical mix.
	- Produces feature importance for explainability.
	Fallback: RandomForest if LightGBM not available.

6. Evaluation Metrics
	MAE (Mean Absolute Error): “average minutes off” prediction.
	R² Score: % of variance explained.
	Evaluated on hold-out test set.
	Diagnostics: print sample rows showing delay_minutes, calculated_delay_minutes, 	traffic_delay_minutes, and total_delay_minutes.

7. Output
	For each row:
	- calculated_delay_minutes → ML correction based on engineering features.
	- traffic_delay_minutes → external correction based on TomTom traffic data.
	- total_delay_minutes = delay_minutes + calculated_delay_minutes + 						traffic_delay_minutes.
	Save enriched dataset as:
	- mbta_delays_with_predictions.csv

In [6]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install --upgrade lightgbm

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install joblib

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


# ML_pipeline

End-to-end: engineering CSV + alerts archive -> residual ML model -> traffic -> final predictions CSV.

Inputs:
 - mbta_delays_with_all_engineering_features.csv
 - Archive_2020-2023.zip (alerts archive)

Outputs:
 - mbta_delays_with_predictions.csv
 - calculated_delay_model.pkl
 - preprocessor.pkl

In [ ]:
import os, glob, zipfile, logging
import numpy as np, pandas as pd, requests, joblib
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")

INPUT_CSV = "mbta_delays_with_all_engineering_features.csv"
ZIP_CANDIDATES = glob.glob("*.zip")
ALERTS_ZIP = next((z for z in ZIP_CANDIDATES if "archive" in z.lower() or "alerts" in z.lower() or "2020-2023" in z), None)
OUTPUT_CSV = "mbta_delays_with_predictions.csv"
MODEL_OUT = "calculated_delay_model.pkl"
PREP_OUT = "preprocessor.pkl"
RANDOM_STATE = 42

ML_SHRINK, RULE_SHRINK, MIN_CLIP, FRACTION_OF_OBSERVED, MAX_ABS_CLIP, RULE_BASE_SHRINK = 0.5, 0.7, 5.0, 0.5, 30.0, 0.8

TOMTOM_API_KEY = os.environ.get("TOMTOM_API_KEY", None)
if not TOMTOM_API_KEY:
    TOMTOM_API_KEY = "UUPZ2IClghy83UBtCRnv2dCAqhzRT8rC" 

FEATURE_COLS = [
    "hour_of_day","day_of_week","is_peak","is_weekend","is_holiday",
    "precipitation_mm","visibility_m","weathercode","route_id_x","stop_id",
    "trip_length","scheduled_runtime","previous_delay"
]
NUMERIC_FEATURES = [
    "hour_of_day","day_of_week","is_peak","is_weekend","is_holiday",
    "precipitation_mm","visibility_m","trip_length","scheduled_runtime",
    "previous_delay","alert_any","alert_count","alert_severity_max"
]
CATEGORICAL_FEATURES = ["weathercode","route_id_x","stop_id"]

def load_engineering_csv(path):
    df = pd.read_csv(path, parse_dates=["actual_arrival","weather_time"], low_memory=False)
    if "delay_minutes" not in df.columns:
        raise RuntimeError("CSV must contain delay_minutes")
    return df

def load_alerts_zip(zip_path):
    if not zip_path: return pd.DataFrame()
    records=[]
    with zipfile.ZipFile(zip_path,"r") as z:
        for fname in z.namelist():
            if fname.endswith(".csv"):
                with z.open(fname) as f:
                    tmp=pd.read_csv(f,low_memory=False)
                    for c in ["alert_id","route_id","stop_id","active_period_start_dt","active_period_end_dt","severity"]:
                        if c not in tmp: tmp[c]=np.nan
                    records.append(tmp[["alert_id","route_id","stop_id","active_period_start_dt","active_period_end_dt","severity"]])
    alerts=pd.concat(records,ignore_index=True) if records else pd.DataFrame()
    for col in ["active_period_start_dt","active_period_end_dt"]:
        if col in alerts: alerts[col]=pd.to_datetime(alerts[col],utc=True,errors="coerce")
    alerts["severity"]=pd.to_numeric(alerts.get("severity"),errors="coerce")
    return alerts

def build_alert_features_vectorized(df,alerts):
    df=df.copy(); df["_row_index"]=np.arange(len(df))
    if alerts.empty:
        df["alert_any"]=0; df["alert_count"]=0; df["alert_severity_max"]=0.0
        return df.drop(columns=["_row_index"])
    route_vals=df["route_id_x"].dropna().unique() if "route_id_x" in df.columns else []
    stop_vals=df["stop_id"].dropna().unique() if "stop_id" in df.columns else []
    cand=alerts[(alerts["route_id"].isin(route_vals)) | (alerts["stop_id"].isin(stop_vals))] if len(route_vals)+len(stop_vals)>0 else alerts.copy()
    if cand.empty:
        df["alert_any"]=0; df["alert_count"]=0; df["alert_severity_max"]=0.0
        return df.drop(columns=["_row_index"])
    if "route_id_x" in df.columns:
        m_route=df.merge(cand[["alert_id","route_id","active_period_start_dt","active_period_end_dt","severity"]],
                         left_on="route_id_x",right_on="route_id",how="left")
        cond_r=(~m_route["active_period_start_dt"].isna()) & (~m_route["actual_arrival"].isna()) & \
               (m_route["actual_arrival"]>=m_route["active_period_start_dt"]) & \
               (m_route["active_period_end_dt"].isna() | (m_route["actual_arrival"]<=m_route["active_period_end_dt"]))
        m_route_valid=m_route.loc[cond_r,["_row_index","alert_id","severity"]]
    else: m_route_valid=pd.DataFrame(columns=["_row_index","alert_id","severity"])
    if "stop_id" in df.columns:
        m_stop=df.merge(cand[["alert_id","stop_id","active_period_start_dt","active_period_end_dt","severity"]],
                        on="stop_id",how="left")
        cond_s=(~m_stop["active_period_start_dt"].isna()) & (~m_stop["actual_arrival"].isna()) & \
               (m_stop["actual_arrival"]>=m_stop["active_period_start_dt"]) & \
               (m_stop["active_period_end_dt"].isna() | (m_stop["actual_arrival"]<=m_stop["active_period_end_dt"]))
        m_stop_valid=m_stop.loc[cond_s,["_row_index","alert_id","severity"]]
    else: m_stop_valid=pd.DataFrame(columns=["_row_index","alert_id","severity"])
    matches=pd.concat([m_route_valid,m_stop_valid],ignore_index=True)
    if matches.empty:
        df["alert_any"]=0; df["alert_count"]=0; df["alert_severity_max"]=0.0
        return df.drop(columns=["_row_index"])
    agg=matches.groupby("_row_index").agg(alert_count=("alert_id","nunique"),alert_severity_max=("severity","max")).reset_index()
    df=df.merge(agg,on="_row_index",how="left")
    df["alert_count"]=df["alert_count"].fillna(0).astype(int)
    df["alert_severity_max"]=df["alert_severity_max"].fillna(0.0)
    df["alert_any"]=(df["alert_count"]>0).astype(int)
    return df.drop(columns=["_row_index"])

def safe_delta(cond,target,min_count=30):
    a,b=target[cond].dropna(),target[~cond].dropna()
    return 0.0 if len(a)<min_count or len(b)<min_count else float(a.mean()-b.mean())

def get_traffic_delay_cache(latlon_pairs,apikey):
    out={p:0.0 for p in latlon_pairs}
    if not apikey: 
        logging.warning("No TomTom API key provided: traffic delays will be zero.")
        return out
    for lat,lon in latlon_pairs:
        try:
            url=f"https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json?point={lat},{lon}&unit=KMPH&key={apikey}"
            r=requests.get(url,timeout=6)
            if r.ok:
                seg=r.json().get("flowSegmentData",{})
                curr,free=seg.get("currentTravelTime"),seg.get("freeFlowTravelTime")
                if curr and free: out[(lat,lon)]=max(0,(curr-free)/60.0)
        except: out[(lat,lon)]=0.0
    return out

def main():
    logging.info("Starting pipeline")
    df=load_engineering_csv(INPUT_CSV)
    df["actual_arrival"]=pd.to_datetime(df["actual_arrival"],utc=True,errors="coerce")
    alerts=load_alerts_zip(ALERTS_ZIP); df=build_alert_features_vectorized(df,alerts)
    for c in FEATURE_COLS:
        if c not in df: df[c]=np.nan
    df["route_hour_mean"]=df.groupby(["route_id_x","hour_of_day"])["delay_minutes"].transform("mean").fillna(df["delay_minutes"].mean())
    df["residual_target"]=df["delay_minutes"]-df["route_hour_mean"]

    numeric=[c for c in NUMERIC_FEATURES if c in df.columns and pd.api.types.is_numeric_dtype(df[c])]
    cat=[c for c in CATEGORICAL_FEATURES if c in df.columns]
    all_features=list(dict.fromkeys(numeric+cat))

    try: ohe=OneHotEncoder(handle_unknown="ignore",sparse_output=False)
    except TypeError: ohe=OneHotEncoder(handle_unknown="ignore",sparse=False)

    preprocessor=ColumnTransformer([
        ("num",SimpleImputer(strategy="median"),numeric),
        ("cat",Pipeline([("imputer",SimpleImputer(strategy="constant",fill_value="__MISSING__")),("ohe",ohe)]),cat)
    ],remainder="drop")

    X,y=df[all_features].copy(),df["residual_target"].values

    if not df["actual_arrival"].isna().all():
        df_sorted=df.sort_values("actual_arrival").reset_index(drop=True); n=len(df_sorted)
        i1,i2=int(n*0.8),int(n*0.9)
        train_idx,val_idx,test_idx=df_sorted.index[:i1],df_sorted.index[i1:i2],df_sorted.index[i2:]
        X_train,X_val,X_test=X.loc[train_idx],X.loc[val_idx],X.loc[test_idx]
        y_train,y_val,y_test=y[train_idx],y[val_idx],y[test_idx]
    else:
        X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=RANDOM_STATE)
        X_val,y_val=X_test,y_test

    prep_pipe=Pipeline([("prep",preprocessor)])
    X_train_prep=prep_pipe.fit_transform(X_train)
    X_val_prep=prep_pipe.transform(X_val)
    X_test_prep=prep_pipe.transform(X_test)
    X_all_prep=prep_pipe.transform(X)
    joblib.dump(prep_pipe,PREP_OUT)

    model=None; used_lgb=False
    try:
        import lightgbm as lgb
        logging.info("Training LightGBM (native)")
        train_data=lgb.Dataset(X_train_prep,label=y_train)
        val_data=lgb.Dataset(X_val_prep,label=y_val,reference=train_data)
        params={"objective":"regression","metric":"l2","learning_rate":0.05,"num_leaves":31,"seed":RANDOM_STATE}
        model=lgb.train(params,train_data,num_boost_round=2000,valid_sets=[val_data])
        used_lgb=True
        y_test_pred=model.predict(X_test_prep)
    except Exception as e:
        logging.warning("LightGBM training failed (%s). Using RandomForest fallback.",e)
        from sklearn.ensemble import RandomForestRegressor
        rf=RandomForestRegressor(n_estimators=300,max_depth=12,random_state=RANDOM_STATE,n_jobs=-1)
        rf.fit(X_train_prep,y_train)
        model=rf; y_test_pred=model.predict(X_test_prep)

    joblib.dump(model,MODEL_OUT)

    logging.info("Test MAE: %.3f",mean_absolute_error(y_test,y_test_pred))
    logging.info("Test R2 : %.3f",r2_score(y_test,y_test_pred))

    train_df=df.loc[train_idx] if 'train_idx' in locals() else df
    d_peak=safe_delta(train_df.get("is_peak")==1,train_df["residual_target"])
    d_precip=safe_delta(train_df.get("precipitation_mm",0).fillna(0)>0.5,train_df["residual_target"])
    d_lowvis=safe_delta(train_df.get("visibility_m",99999).fillna(99999)<1000,train_df["residual_target"])
    d_alert=safe_delta(train_df.get("alert_any")==1,train_df["residual_target"])
    d_peak*=RULE_BASE_SHRINK; d_precip*=RULE_BASE_SHRINK; d_lowvis*=RULE_BASE_SHRINK; d_alert*=RULE_BASE_SHRINK

    rule_adj=(df["is_peak"].fillna(0).astype(int)*d_peak+
              (df["precipitation_mm"].fillna(0)>0.5).astype(int)*d_precip+
              (df["visibility_m"].fillna(99999)<1000).astype(int)*d_lowvis+
              (df["alert_any"].fillna(0).astype(int)*d_alert))

    if used_lgb: ml_pred_all=model.predict(X_all_prep)
    else: ml_pred_all=model.predict(X_all_prep)

    raw_comb=(ML_SHRINK*ml_pred_all)+(RULE_SHRINK*rule_adj.values)
    observed_abs=np.abs(df["delay_minutes"].values)
    clip_bounds=np.minimum(MAX_ABS_CLIP,np.maximum(MIN_CLIP,FRACTION_OF_OBSERVED*(observed_abs+1e-9)))
    calc_adj=np.clip(raw_comb,-clip_bounds,clip_bounds)
    near_zero=(np.abs(df["delay_minutes"])<1.0); calc_adj[near_zero]=np.clip(calc_adj[near_zero],-MIN_CLIP/2,MIN_CLIP/2)
    df["ml_raw_residual_prediction"],df["rule_adjustment"],df["calculated_delay_minutes"]=ml_pred_all,rule_adj,np.nan_to_num(calc_adj)

    if "latitude" in df.columns and "longitude" in df.columns:
        coords=df[["latitude","longitude"]].dropna().apply(lambda r:(round(float(r["latitude"]),6),round(float(r["longitude"]),6)),axis=1)
        coord_to_delay=get_traffic_delay_cache(sorted(set(coords.tolist())),TOMTOM_API_KEY)
        df["traffic_delay_minutes"]=df.apply(lambda r:coord_to_delay.get((round(float(r["latitude"]),6),round(float(r["longitude"]),6)),0.0),axis=1)
    else: df["traffic_delay_minutes"]=0.0

    df["total_delay_minutes"]=df["delay_minutes"].fillna(0)+df["calculated_delay_minutes"].fillna(0)+df["traffic_delay_minutes"].fillna(0)
    df.to_csv(OUTPUT_CSV,index=False); logging.info("Saved to %s (rows=%d)",OUTPUT_CSV,len(df))

if __name__=="__main__":
    main()

2025-11-09 00:28:33,406 INFO Starting pipeline


2025-11-09 00:28:48,615 INFO Training LightGBM (native)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000513 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 64
[LightGBM] [Info] Number of data points in the train set: 73, number of used features: 6
[LightGBM] [Info] Start training from score 0.044178
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


2025-11-09 00:28:48,938 INFO Test MAE: 1.321


2025-11-09 00:28:48,940 INFO Test R2 : -0.036


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

2025-11-09 00:29:08,041 INFO Saved to mbta_delays_with_predictions.csv (rows=92)


In [1]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


# Evaluation

evaluate_predictions_no_args.py

Evaluates mbta_delays_with_predictions.csv (or any predictions CSV) and produces:
 - Regression metrics (MAE, RMSE, R2, MAPE, ExplainedVariance)
 - Diagnostic plots (saved to / shown inline)
 - Per-route and per-stop MAE tables

Outputs are saved to: preds_evaluation/

In [ ]:
import os, math, logging
from collections import OrderedDict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score

INPUT_CSV = "mbta_delays_with_predictions.csv"   
OUT_DIR = "preds_evaluation"

PRED_CANDIDATES = [
    "ml_raw_residual_prediction", "ml_pred_residual", "ml_pred",
    "predicted_residual", "ml_prediction", "pred_residual"
]

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")

def load_csv_guess_dates(path):
    sample = pd.read_csv(path, nrows=0)
    parse_dates = [c for c in ["actual_arrival", "weather_time"] if c in sample.columns]
    return pd.read_csv(path, parse_dates=parse_dates, low_memory=False)

def find_prediction_column(df):
    for c in PRED_CANDIDATES:
        if c in df.columns:
            logging.info(f"Using prediction column: {c}")
            return c
    for c in df.columns:
        lc = c.lower()
        if ("ml" in lc or "pred" in lc or "prediction" in lc) and ("resid" in lc or "residual" in lc):
            logging.info(f"Using fuzzy-match prediction column: {c}")
            return c
    raise RuntimeError(f"No prediction column found. Expected one of: {PRED_CANDIDATES}")

def reconstruct_route_hour_mean_if_missing(df):
    if "route_hour_mean" in df.columns:
        return df
    if {"route_id_x", "hour_of_day", "delay_minutes"} <= set(df.columns):
        df["route_hour_mean"] = df.groupby(["route_id_x", "hour_of_day"])["delay_minutes"].transform("mean").fillna(df["delay_minutes"].mean())
        logging.info("Reconstructed route_hour_mean from route_id_x + hour_of_day.")
    else:
        df["route_hour_mean"] = df["delay_minutes"].mean()
        logging.warning("Could not reconstruct route_hour_mean; used global mean.")
    return df

def compute_residual_target(df):
    if "residual_target" in df.columns:
        return df
    df = reconstruct_route_hour_mean_if_missing(df)
    df["residual_target"] = df["delay_minutes"] - df["route_hour_mean"]
    logging.info("Computed residual_target = delay_minutes - route_hour_mean")
    return df

def time_based_split_indices(df):
    if "actual_arrival" in df.columns and df["actual_arrival"].notna().any():
        df_sorted = df.reset_index().sort_values("actual_arrival")
        n = len(df_sorted)
        i1, i2 = int(n*0.8), int(n*0.9)
        return (df_sorted.loc[:i1-1,"index"].tolist(),
                df_sorted.loc[i1:i2-1,"index"].tolist(),
                df_sorted.loc[i2:,"index"].tolist())
    return None

def regression_metrics(y_true, y_pred):
    mask = ~np.isnan(y_true) & ~np.isnan(y_pred)
    if not np.any(mask): return {}
    y_true, y_pred = y_true[mask], y_pred[mask]
    mae = mean_absolute_error(y_true, y_pred)
    rmse = math.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    ev = explained_variance_score(y_true, y_pred)
    with np.errstate(divide='ignore', invalid='ignore'):
        mape = np.nanmean(np.abs((y_true - y_pred) / np.where(np.abs(y_true) < 1e-9, np.nan, y_true))) * 100
    return dict(MAE=mae, RMSE=rmse, R2=r2, ExplainedVariance=ev, MAPE_percent=mape, N=len(y_true))

def save_plot(fig, name):
    path = os.path.join(OUT_DIR, name)
    fig.savefig(path, dpi=150, bbox_inches="tight")
    plt.close(fig)
    logging.info(f"Saved plot: {path}")

def evaluate_predictions(input_csv=INPUT_CSV, out_dir=OUT_DIR):
    os.makedirs(out_dir, exist_ok=True)
    logging.info(f"Loading {input_csv}")
    df = load_csv_guess_dates(input_csv)

    pred_col = find_prediction_column(df)
    df = compute_residual_target(df)

    df[pred_col] = pd.to_numeric(df[pred_col], errors="coerce")
    df["residual_target"] = pd.to_numeric(df["residual_target"], errors="coerce")

    tb = time_based_split_indices(df)
    if tb:
        train_idx, val_idx, test_idx = tb
        logging.info(f"Time split: train={len(train_idx)}, val={len(val_idx)}, test={len(test_idx)}")
    else:
        train_idx = val_idx = test_idx = []

    metrics = OrderedDict()
    metrics["overall"] = regression_metrics(df["residual_target"].values, df[pred_col].values)
    if tb:
        metrics["train"] = regression_metrics(df.loc[train_idx, "residual_target"], df.loc[train_idx, pred_col])
        metrics["val"]   = regression_metrics(df.loc[val_idx, "residual_target"], df.loc[val_idx, pred_col])
        metrics["test"]  = regression_metrics(df.loc[test_idx, "residual_target"], df.loc[test_idx, pred_col])

    metrics_df = pd.DataFrame.from_dict(metrics, orient="index")
    metrics_df.to_csv(os.path.join(out_dir, "metrics_overall_and_splits.csv"))
    logging.info(f"Saved metrics to {out_dir}/metrics_overall_and_splits.csv\n{metrics_df}")

    df.to_csv(os.path.join(out_dir, "predictions_with_residual_target.csv"), index=False)

    if tb and len(test_idx) > 0:
        subset = df.loc[test_idx]
        tag, title_suffix = "test", " (TEST)"
    else:
        subset = df
        tag, title_suffix = "all", " (ALL)"

    x = subset["residual_target"].values
    y = subset[pred_col].values
    resid = x - y

    fig = plt.figure(figsize=(6,6))
    plt.scatter(x, y, s=10, alpha=0.4)
    mn, mx = np.nanmin(np.concatenate([x,y])), np.nanmax(np.concatenate([x,y]))
    plt.plot([mn,mx],[mn,mx],"--")
    plt.xlabel("Actual residual (min)")
    plt.ylabel("Predicted residual (min)")
    plt.title("Pred vs Actual Residuals" + title_suffix)
    save_plot(fig, f"pred_vs_actual_{tag}.png")

    fig = plt.figure(figsize=(7,4))
    plt.hist(resid[~np.isnan(resid)], bins=80)
    plt.xlabel("Residual error (actual - pred)")
    plt.title("Residual Error Distribution" + title_suffix)
    save_plot(fig, f"residual_hist_{tag}.png")

    fig = plt.figure(figsize=(6,4))
    plt.scatter(y, resid, s=8, alpha=0.4)
    plt.axhline(0, ls="--")
    plt.xlabel("Predicted residual")
    plt.ylabel("Error (actual - pred)")
    plt.title("Residuals vs Predicted" + title_suffix)
    save_plot(fig, f"residuals_vs_pred_{tag}.png")

    if "hour_of_day" in df.columns:
        tmp = df.copy()
        tmp["abs_err"] = np.abs(tmp["residual_target"] - tmp[pred_col])
        agg = tmp.groupby("hour_of_day")["abs_err"].mean().reset_index()
        fig = plt.figure(figsize=(8,4))
        plt.plot(agg["hour_of_day"], agg["abs_err"], marker="o")
        plt.xlabel("Hour of Day")
        plt.ylabel("MAE (minutes)")
        plt.title("MAE by Hour of Day")
        save_plot(fig, "mae_by_hour.png")

    if "route_id_x" in df.columns:
        route_mae = df.groupby("route_id_x").apply(lambda g: mean_absolute_error(g["residual_target"], g[pred_col])).reset_index()
        route_mae.columns = ["route_id_x","MAE_residual"]
        route_mae.sort_values("MAE_residual", ascending=False).to_csv(os.path.join(out_dir,"per_route_mae.csv"), index=False)
    if "stop_id" in df.columns:
        stop_mae = df.groupby("stop_id").apply(lambda g: mean_absolute_error(g["residual_target"], g[pred_col])).reset_index()
        stop_mae.columns = ["stop_id","MAE_residual"]
        stop_mae.sort_values("MAE_residual", ascending=False).to_csv(os.path.join(out_dir,"per_stop_mae.csv"), index=False)

    logging.info("Evaluation complete. All results saved in: %s", out_dir)
    return metrics_df

try:
    metrics_df = evaluate_predictions(INPUT_CSV, OUT_DIR)
    display(metrics_df)
except Exception as e:
    logging.exception("Evaluation failed: %s", e)

2025-11-09 00:30:51,632 INFO Loading mbta_delays_with_predictions.csv
2025-11-09 00:30:51,660 INFO Using prediction column: ml_raw_residual_prediction
2025-11-09 00:30:51,671 INFO Time split: train=54, val=0, test=72
2025-11-09 00:30:51,702 INFO Saved metrics to preds_evaluation/metrics_overall_and_splits.csv
              MAE      RMSE        R2  ExplainedVariance  MAPE_percent   N
overall  1.448835  1.922121  0.595182           0.595256    230.131478  92
train    1.287856  1.704922  0.713573           0.722635    334.101304  54
test     1.490206  1.986870  0.309505           0.309795    269.831025  72
2025-11-09 00:30:51,920 INFO Saved plot: preds_evaluation\pred_vs_actual_test.png
2025-11-09 00:30:52,067 INFO Saved plot: preds_evaluation\residual_hist_test.png
2025-11-09 00:30:52,174 INFO Saved plot: preds_evaluation\residuals_vs_pred_test.png
2025-11-09 00:30:52,285 INFO Saved plot: preds_evaluation\mae_by_hour.png
C:\Users\karti\AppData\Local\Temp\ipykernel_24240\1714826420.py:188

,MAE,RMSE,R2,ExplainedVariance,MAPE_percent,N
overall,1.448835,1.922121,0.595182,0.595256,230.131478,92
train,1.287856,1.704922,0.713573,0.722635,334.101304,54
test,1.490206,1.986870,0.309505,0.309795,269.831025,72


In [12]:
from datetime import datetime
current_datetime = datetime.now()
print(current_datetime)

2025-11-09 00:29:08.783123
